### Partitioning
Partitioning is a generic term for dividing data across tables or databases. We can partition vertically or horizontally as shown:  
<img src="https://assets.digitalocean.com/articles/understanding_sharding/DB_image_1_cropped.png" width="650" height="auto" />

Horizontal partitioning is called **sharding** and it is the one that is supported by MySQL (no vertical partitioning in MySQL).  

Advantages:
- now able to accomodate large amount of data by splitting it across machines
- better reselience in case of failure
- faster query in some cases

### Sharding Methods
**Hash based** one of the column is hashed and the hash result is used to distribute rows:   
<img src="https://assets.digitalocean.com/articles/understanding_sharding/DB_image_2_cropped.png" width="650" height="auto" />  
The problem with this is if we add new machines then how to we distribute now? We may have to update our hashing logic, redistribute all data, etc

In MySQL we can define hash based partitioning in the following manner:
```SQL
-- Only InnoDB and NDB engines support partitioning
CREATE TABLE ti (id INT, amount DECIMAL(7,2), tr_date DATE)
ENGINE=INNODB
PARTITION BY HASH( MONTH(tr_date) )
PARTITIONS 6;
```

**Range based**  
<img src="https://assets.digitalocean.com/articles/understanding_sharding/DB_image_3_cropped.png" width="650" height="auto" />  
The disadvantage is that partitions can grow non-uniformly.
```SQL
CREATE TABLE members (
    firstname VARCHAR(25) NOT NULL,
    lastname VARCHAR(25) NOT NULL,
    username VARCHAR(16) NOT NULL,
    email VARCHAR(35),
    joined DATE NOT NULL
)
PARTITION BY RANGE( YEAR(joined) ) (
    PARTITION p0 VALUES LESS THAN (1960),
    PARTITION p1 VALUES LESS THAN (1970),
    PARTITION p2 VALUES LESS THAN (1980),
    PARTITION p3 VALUES LESS THAN (1990),
    PARTITION p4 VALUES LESS THAN MAXVALUE
);
```

**List based** if the column value (using which we are partitioning) has discrete values.
```SQL
CREATE TABLE t2 (val INT)
PARTITION BY LIST(val)(
PARTITION p0 VALUES IN (1,3,5),
PARTITION p1 VALUES IN (2,4,6)
```